# Notebook to produce all plots for first year presentations (EGU2023, CPT2023, M2Lines2023)

In [ ]:
import xarray as xr
import xgcm
from matplotlib import pyplot as plt
import nc_time_axis

#plt.rcParams['axes.facecolor']='black'
#plt.style.use('dark_background')
plt.style.use('default')

plt.rcParams.update({'font.size': 20})

In [ ]:
path   = "/data/dkamm/nemo_output/NEVERWORLD/"

## Load data (lazy)

In [ ]:
n_restarts = 5      # Number of restarts

In [ ]:
V = xr.open_dataset(path + 'restart0/NEVERWORLD_grid_V.nc')
U = xr.open_dataset(path + 'restart0/NEVERWORLD_grid_U.nc')
T = xr.open_dataset(path + 'restart0/NEVERWORLD_grid_T.nc')
for i in range(1,n_restarts):
    _path = path + 'restart{}/'.format(i)
    _V = xr.open_dataset(_path + 'NEVERWORLD_grid_V.nc')
    _U = xr.open_dataset(_path + 'NEVERWORLD_grid_U.nc')
    _T = xr.open_dataset(_path + 'NEVERWORLD_grid_T.nc')
    V = xr.concat([V.chunk(), _V.chunk()], "time_counter")
    U = xr.concat([U.chunk(), _U.chunk()], "time_counter")
    T = xr.concat([T.chunk(), _T.chunk()], "time_counter")
    

In [ ]:
n_restarts_hr = 2

In [ ]:
V_hr = xr.open_dataset(path + 'restart10/NEVERWORLD_grid_V.nc')
U_hr = xr.open_dataset(path + 'restart10/NEVERWORLD_grid_U.nc')
T_hr = xr.open_dataset(path + 'restart10/NEVERWORLD_grid_T.nc')
for i in range(n_restarts + 1,n_restarts + n_restarts_hr):
    _path = path + 'restart{}/'.format(i)
    _V_hr = xr.open_dataset(_path + 'NEVERWORLD_grid_V.nc')
    _U_hr = xr.open_dataset(_path + 'NEVERWORLD_grid_U.nc')
    _T_hr = xr.open_dataset(_path + 'NEVERWORLD_grid_T.nc')
    V_hr = xr.concat([V_hr.chunk(), _V_hr.chunk()], "time_counter")
    U_hr = xr.concat([U_hr.chunk(), _U_hr.chunk()], "time_counter")
    T_hr = xr.concat([T_hr.chunk(), _T_hr.chunk()], "time_counter")

## Load domain (not lazy)

In [ ]:
from pathlib import Path
import xnemogcm as xn

In [ ]:
datadir_no_ridge = Path(path + 'NO_RIDGE_init/')

In [ ]:
domcfg_no_ridge = xn.open_domain_cfg(datadir=datadir_no_ridge)

In [ ]:
datadir_1deg = Path(path + 'restart9')
datadir_1_4deg_old = Path(path + 'restart10_1')
datadir_1_4deg = Path(path + 'restart11')

In [ ]:
domcfg_1deg = xn.open_domain_cfg(datadir=datadir_1deg)
domcfg_1_4deg = xn.open_domain_cfg(datadir=datadir_1_4deg)
domcfg_1_4deg_old = xn.open_domain_cfg(datadir=datadir_1_4deg_old)

## Bathymetry

In [ ]:
import cartopy.crs as ccrs

bathy = domcfg_1_4deg

## Barotropic streamfunction

In [ ]:
Data_1deg = xn.open_nemo(domcfg=domcfg_1deg, files=[
        datadir_1deg / 'NEVERWORLD_grid_T.nc',
        datadir_1deg / 'NEVERWORLD_grid_U.nc',
        datadir_1deg / 'NEVERWORLD_grid_V.nc'
    ])

In [ ]:
Data_1deg

### Interpolating uoce on y_f

In [ ]:
metrics = {
    ('X',): ['e1t', 'e1u', 'e1v', 'e1f'], # X distances
    ('Y',): ['e2t', 'e2u', 'e2v', 'e2f'], # Y distances
    ('Z',): ['e3t_0', 'e3u_0', 'e3v_0', 'e3f_0'] # Z distances
}

grid_1deg = xgcm.Grid(domcfg_1deg,
        coords={
            "X": {"right": "x_f", "center": "x_c"},
            "Y": {"right": "y_f", "center":"y_c"},
            "Z": {"right": "z_f", "center": "z_c"}
        },
        periodic=False,
        metrics=metrics
)

In [ ]:
Data_1deg['u_f'] = grid_1deg.interp(Data_1deg.uoce, 'Y')

### Integrate over depth

In [ ]:
Data_1deg['U'] = grid_1deg.integrate(Data_1deg.u_f, 'Z')

### Cummulative integral in i-direction at last time step

In [ ]:
Psi = (Data_1deg.U[-1,::-1,:] * domcfg_1deg.e2f[::-1,:]).cumsum('y_f') / 1e6

In [ ]:
# U_s_grid = U_s_grid.assign_coords({'x_f': U_s_grid.x_f.values - 60.})
fig, axs = plt.subplots(1,1,figsize=(8,6))
a = Psi.plot.contourf(x='x_f', y='y_f', cmap='RdBu_r', levels=30, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\psi$ [Sv]')
axs.set_ylabel('latitude [°N]')
axs.set_xlabel('longitude [°E]')
plt.title('')

## Barotropic streamfunction (1/4 degree)

In [ ]:
Data_1_4deg = xn.open_nemo(domcfg=domcfg_1_4deg, files=[
        datadir_1_4deg / 'NEVERWORLD_grid_T.nc',
        datadir_1_4deg / 'NEVERWORLD_grid_U.nc',
        datadir_1_4deg / 'NEVERWORLD_grid_V.nc'
    ])

### Interpolating uoce on y_f

In [ ]:
metrics = {
    ('X',): ['e1t', 'e1u', 'e1v', 'e1f'], # X distances
    ('Y',): ['e2t', 'e2u', 'e2v', 'e2f'], # Y distances
    ('Z',): ['e3t_0', 'e3u_0', 'e3v_0', 'e3f_0'] # Z distances
}

grid_1_4deg = xgcm.Grid(domcfg_1_4deg,
        coords={
            "X": {"right": "x_f", "center": "x_c"},
            "Y": {"right": "y_f", "center":"y_c"},
            "Z": {"right": "z_f", "center": "z_c"}
        },
        periodic=False,
        metrics=metrics
)

In [ ]:
Data_1_4deg['u_f'] = grid_1_4deg.interp(Data_1_4deg.uoce, 'Y')

### Integrate over depth

In [ ]:
Data_1_4deg['U'] = grid_1_4deg.integrate(Data_1_4deg.u_f, 'Z')

### Cummulative integral in i-direction at last time step

In [ ]:
Psi_1_4deg = (Data_1_4deg.U[0,::-1,:] * domcfg_1_4deg.e2f[::-1,:]).cumsum('y_f') / 1e6

In [ ]:
# U_s_grid = U_s_grid.assign_coords({'x_f': U_s_grid.x_f.values - 60.})
fig, axs = plt.subplots(1,1,figsize=(8,6))
a = Psi_1_4deg.plot.contourf(x='x_f', y='y_f', cmap='RdBu_r', levels=30, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\psi$ [Sv]')
axs.set_ylabel('latitude [°N]')
axs.set_xlabel('longitude [°E]')
plt.title('')

## ACC strength time series

In [ ]:
Data_1deg_500y = Data_1deg.copy()
for i in range(8,-1, -1):
    _datadir = Path(path + 'restart{}/'.format(i))
    _Data = xn.open_nemo(domcfg=domcfg_1deg, files=[
        _datadir / 'NEVERWORLD_grid_T.nc',
        _datadir / 'NEVERWORLD_grid_U.nc',
        _datadir / 'NEVERWORLD_grid_V.nc'
    ])
    Data_1deg_500y = xr.concat([_Data.chunk(), Data_1deg_500y.chunk()], "t")

In [ ]:
ACC_1deg = Data_1deg_500y.isel(x_f=0).uocetr_eff.sum(['y_c', 'z_c']) / 1e6

In [ ]:
ACC_1deg

In [ ]:
for i in range(10,9, -1):
    _datadir = Path(path + 'restart{}/'.format(i))
    _Data = xn.open_nemo(domcfg=domcfg_1_4deg, files=[
        _datadir / 'NEVERWORLD_grid_T.nc',
        _datadir / 'NEVERWORLD_grid_U.nc',
        _datadir / 'NEVERWORLD_grid_V.nc'
    ])
    Data_1_4deg = xr.concat([_Data.chunk(), Data_1_4deg.chunk()], "t")

In [ ]:
ACC_1_4deg = Data_1_4deg.isel(x_f=0).uocetr_eff.sum(['y_c', 'z_c']) / 1e6

In [ ]:
ACC = xr.concat([ACC_1deg.chunk(), ACC_1_4deg.chunk()], "t")

In [ ]:
import cftime as cft
ACC = ACC.assign_coords(dict({'t':cft.to_tuple(ACC.indexes['t'])[0]}))

In [ ]:
ACC.data

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(12,6))
a = plt.plot(ACC.t.values[:500],ACC.data[:500],color='black', label='1° resolution')
a = plt.plot(ACC.t.values[500:],ACC.data[500:],color='darkred', label='1/4° resolution')
axs.set_ylabel('ACC transport [Sv]')
plt.title('')
plt.xlabel('time [years]')
plt.grid()
plt.legend()

axes = plt.axes([.45, .4, .40, .2])
axes.plot(ACC.t.values[494:500], ACC.data[494:500], c='black', lw=1)
axes.plot(ACC.t.values[500:], ACC.data[500:], c='darkred', lw=1)
axes.plot(ACC.t.values[499:501], ACC.data[499:501], c='grey', linestyle=':', lw=1)

axs.set_facecolor('none')
fig.savefig('ACC.png', transparent=True)

## Meridional overturning

In [ ]:
import gsw as gsw

### Cutting the bottom cell and create a new grid object

In [ ]:
import cf_xarray

Data_1deg_top = Data_1deg_500y.isel(z_c=slice(0,-1)).copy()

level_outer_data = cf_xarray.bounds_to_vertices(Data_1deg_top.deptht_bounds.isel(t=-1), 'axis_nbounds').load().data

Data_1deg_top = Data_1deg_top.assign_coords({'z_f': level_outer_data})

grid_1deg_top = xgcm.Grid(Data_1deg_top,
    coords={
        "X": {"center": "x_c"},
        "Y": {"right": "y_f", "center":"y_c"},
        "Z": {"center": "z_c", "outer": "z_f"}
    },
    periodic=False)

### Compute potential density, referenced to 2000m

In [ ]:
Data_1deg_top['sigma0'] = gsw.sigma0(Data_1deg_top.soce, gsw.CT_from_pt(Data_1deg_top.soce, Data_1deg_top.toce))

### Interpolate sigma2 on the cell faces

In [ ]:
sigma_v = grid_1deg_top.interp(Data_1deg_top.sigma0, 'Y')
sigma_outer = grid_1deg_top.interp(sigma_v, 'Z',  boundary='extend')

### Target values for the new density coordinates

In [ ]:
import numpy as np
sigma_target = np.array([24, 24.2, 24.4, 24.65, 24.9, 25.15, 25.4, 25.65,
                        25.9, 26.15, 26.4, 26.57, 26.75, 26.90, 27.05, 27.17,
                        27.30, 27.37, 27.45, 27.52, 27.58,
                        27.63, 27.68, 27.72, 27.75, 27.78, 27.80, 27.815,
                        27.83, 27.845, 27.86, 27.875, 27.89,
                        27.905, 27.92, 27.935, 27.95, 27.965,
                        27.98, 27.99, 28.01, 28.025,
                        28.04, 28.055, 28.07, 28.085, 28.1, 28.35,
                        28.6, 28.8, 29.1])

### Transforming the meridional transport on density coordinates

In [ ]:
vtra_transformed = grid_1deg_top.transform(Data_1deg_top.vocetr_eff,
        'Z',
        sigma_target,
        method='conservative',
        target_data=sigma_outer)

### Computing the meridional overturning

In [ ]:
Over_1deg    = vtra_transformed.isel(t=-1).sum(dim='x_c')[:,::-1].cumsum('sigma0') / 1e6

In [ ]:
Over_1deg = Over_1deg.assign_coords(dict({'y_f': domcfg_1deg.gphif.isel(x_f=10).values}))

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(8,6))
a = (-Over_1deg).plot.contourf(x='y_f', y='sigma0', cmap='RdBu_r', levels=30, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\psi$ [Sv]')
plt.xlabel('latitude [°N]')
axs.invert_yaxis()
plt.ylabel(r'$\sigma_{0}$ [ $kg$ / $m^3$ - 1000 ]')
plt.title('')
fig.savefig('MOC.png', transparent=True)

## Meridional overturning (1/4 degree)

### Cutting the bottom cell and create a new grid object

In [ ]:
Data_1_4deg_top = Data_1_4deg.isel(z_c=slice(0,-1)).copy()

level_outer_data = cf_xarray.bounds_to_vertices(Data_1_4deg_top.deptht_bounds, 'axis_nbounds').load().data

Data_1_4deg_top = Data_1_4deg_top.assign_coords({'z_f': level_outer_data})

grid_1_4deg_top = xgcm.Grid(Data_1_4deg_top,
    coords={
        "X": {"center": "x_c"},
        "Y": {"right": "y_f", "center":"y_c"},
        "Z": {"center": "z_c", "outer": "z_f"}
    },
    periodic=False)

### Compute potential density, referenced to 2000m

In [ ]:
Data_1_4deg_top['sigma0'] = gsw.sigma0(Data_1_4deg_top.soce, gsw.CT_from_pt(Data_1_4deg_top.soce, Data_1_4deg_top.toce))

### Interpolate sigma2 on the cell faces

In [ ]:
sigma_v_1_4 = grid_1_4deg_top.interp(Data_1_4deg_top.sigma0, 'Y')
sigma_outer_1_4 = grid_1_4deg_top.interp(sigma_v_1_4, 'Z',  boundary='extend')

### Transforming the meridional transport on density coordinates

In [ ]:
vtra_transformed_1_4 = grid_1_4deg_top.transform(Data_1_4deg_top.vocetr_eff,
        'Z',
        sigma_target,
        method='conservative',
        target_data=sigma_outer_1_4)

### Computing the meridional overturning

In [ ]:
Over_1_4deg    = vtra_transformed_1_4.isel(t=-1).sum(dim='x_c')[:,::-1].cumsum('sigma0') / 1e6

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(8,6))
a = (-Over_1_4deg).plot.contourf(x='y_f', y='sigma0', cmap='RdBu_r', levels=30, add_colorbar=False)
cbar1 = fig.colorbar(a, ax=axs, label=r'$\psi$ [Sv]')
plt.xlabel('latitude [°N]')
axs.invert_yaxis()
plt.ylabel(r'$\sigma_{0}$ [ $kg$ / $m^3$ - 1000 ]')

## Maximum overturning streamfunction

In [ ]:
def SEOS(ds):
    """Simplified equation of state."""
    params = dict({'a_0': 0.165, 'b_0': 0.76554, 'lambda_1': 0.06, 'mu_1': 0.0001497 ,'rho_0': 1.026 })
    T_a = ds['toce'] - 10.
    S_a = ds['soce'] - 35. 
    z   = ds.e3t.cumsum('z_c')
    rho_a = (((-params['a_0'] * (1 + 0.5 * params['lambda_1'] * T_a + params['mu_1'] * z) * T_a + params['b_0'] * S_a) / params['rho_0'] ) + 1 ) * params['rho_0'] 
    return rho_a

In [ ]:
rho_a = SEOS(Data_1deg)

In [ ]:
rho_a.isel(t=-1, x_c = 25, y_c=slice(1,-1), z_c=slice(0,-1)).plot()

In [ ]:
params = dict({'a_0': 0.165, 'b_0': 0.76554, 'lambda_1': 0.06, 'mu_1': 0.0001497 ,'rho_0': 1.026 })

In [ ]:
Data_1deg_top['sigma0'] = gsw.sigma0(Data_1deg_top.soce, gsw.CT_from_pt(Data_1deg_top.soce, Data_1deg_top.toce))

# Low scottia arc

In [ ]:
domcfg_SA_low.e3t_0.sum('z_c')

In [ ]:
Data_no_ridge = xn.open_nemo(domcfg=domcfg_SA_low, files=[
        datadir_SA_low / 'NEVERWORLD_grid_T.nc',
        datadir_SA_low / 'NEVERWORLD_grid_U.nc',
        datadir_SA_low / 'NEVERWORLD_grid_V.nc'
    ])

In [ ]:
for i in range(1,n_restarts, 1):
    _datadir = Path(path + 'restart{}/'.format(i))
    _Data = xn.open_nemo(domcfg=domcfg_SA_low, files=[
        _datadir / 'NEVERWORLD_grid_T.nc',
        _datadir / 'NEVERWORLD_grid_U.nc',
        _datadir / 'NEVERWORLD_grid_V.nc'
    ])
    Data_SA_low = xr.concat([Data_SA_low.chunk(), _Data.chunk()], "t")

In [ ]:
import cftime as cft
Data_SA_low = Data_SA_low.assign_coords(dict({'t':cft.to_tuple(Data_SA_low.indexes['t'])[0]}))

In [ ]:
ACC_low = Data_SA_low.isel(x_f=0).uocetr_eff.sum(['y_c', 'z_c']) / 1e6

In [ ]:
ACC_low.plot()